In [1]:
import os
import sys

os.chdir('..')

parent_directory = os.getcwd()
sys.path.append(parent_directory)

In [2]:
USER_AGENTS = [
  "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Mobile Safari/537.3",
  "Mozilla/5.0 (Linux; Android 13; SAMSUNG SM-A326B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/23.0 Chrome/115.0.0.0 Mobile Safari/537.3",
  "Mozilla/5.0 (Linux; Android 13; SAMSUNG SM-G780G) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/23.0 Chrome/115.0.0.0 Mobile Safari/537.3",
  "Mozilla/5.0 (iPhone; CPU iPhone OS 17_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/121.0.6167.171 Mobile/15E148 Safari/604",
  "Mozilla/5.0 (Linux; Android 11; moto e20 Build/RONS31.267-94-14) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.178 Mobile Safari/537.3",
  "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Mobile Safari/537.3",
  "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Mobile/15E148 Safari/604.",
  "Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36",
  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/37.0.2062.94 Chrome/37.0.2062.94 Safari/537.36",
  "Mozilla/5.0 (Linux; Android 10; SM-G996U Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Mobile Safari/537.36",
  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36",
]

PROXIES = [
  "207.148.71.74:80",
  "69.171.76.99:8081",
  "154.203.132.49:8080",
  "1.162.12.227:80",
  "68.178.203.69:8899",
  "43.156.166.9:8899",
  "212.107.28.120:80",
  "47.242.47.64:8888",
  "111.247.40.237:80",
  "133.18.234.13:80",
  "185.217.143.96:80",
  "41.207.242.62:80",
  "49.13.51.71:80",
  "139.59.1.14:8080",
  "217.182.55.226:80",
  "31.11.143.83:80",
  "198.49.68.80:80",
  "45.92.108.112:80",
  "175.139.233.76:80",
  "204.199.68.201:53281",
  "93.103.221.133:80",
  "109.123.80.30:9480",
  "51.89.14.70:80",
]

In [3]:
# Proxy
from src.rea_beta_backend_toolsets.services.proxies.proxy import ProxyRotation

proxy_rotation_manager = ProxyRotation(PROXIES)
proxy_rotation_manager.valid_proxies

[]

In [4]:
# User agents
from src.rea_beta_backend_toolsets.services.user_agents.user_agent import UserAgentRotation

user_agent_rotation_manager = UserAgentRotation(USER_AGENTS)
user_agent_rotation_manager.valid_user_agents

['Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
 'Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Mobile Safari/537.3',
 'Mozilla/5.0 (Linux; Android 13; SAMSUNG SM-A326B) AppleWebKit/537.36 (KHTML, like Gecko) SamsungBrowser/23.0 Chrome/115.0.0.0 Mobile Safari/537.3',
 'Mozilla/5.0 (iPhone; CPU iPhone OS 17_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Mobile/15E148 Safari/604.',
 'Mozilla/5.0 (Linux; Android 10; SM-G996U Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Mobile Safari/537.36',
 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.85 Safari/537.36',
 'Mozilla/5.0 (Linux; Android 11; moto e20 Build/RONS31.267-94-14) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.178 Mobile Safari/537.3',
 'Mozilla/5.0 (iPhon

In [5]:
import requests # type: ignore
from requests.exceptions import RequestException
from typing import Optional
from requests.exceptions import RequestException

def scrape(url: str, headers: Optional[dict] = None, params: Optional[dict] = None) -> Optional[requests.Response]:
    try:
        proxy = proxy_rotation_manager.get_next_proxy().proxy
        

        
        user_agent = user_agent_rotation_manager.get_next_user_agent()
        
        if headers is None:
            headers = {}
        
        headers.update({"User-Agent": user_agent})
        
        proxies = {
            "http": f"http://{proxy}",
            "https": f"http://{proxy}"
        }
        
        response = requests.get(url, headers=headers, proxies=proxies, params=params, timeout=5)
        
        response.raise_for_status()
        
        return response
    
    except (RequestException, ValueError) as e:
        print(f"An error occurred: {e}")
        return None



top_k = 5
country = ""
search = "Bagyo"

base_url = "https://www.google.com/search?q="
query = f"{search}&num={top_k}{f'&gl={country}' if country != '' else ''}"
url = base_url + query 
scrape(url)

An error occurred: No proxies available
